In [1]:
import __init__
from __init__ import DATA_PATH
from __init__ import PACKAGE_PATH
import numpy as np
import pandas as pd
import os
import matplotlib as mplt
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
data = pd.read_csv(os.path.join(DATA_PATH,'BOD_biclass_139rdkit.csv'))
data_139 = data.iloc[:,2:141]
data_train = data_139.values
data_train.shape

(1063, 139)

In [ ]:
from dimreducer import DeepAutoEncoder
